In [1]:
from datetime import datetime
import os
import sys
dirname = os.path.dirname('__file__')

sys.path.append(os.path.join(dirname, "pkg"))
sys.path.append(os.path.join(dirname, "pkg", "preproc"))
from pkg.preproc import base

#### Objectives

1. Identify number of potential speakers (clustering)
    1.1 Use Factors: Readability (F-K), doc_length, n_mentions, n_hashtags, time_of_day??, avg_word_size, avg_num_syllables
2. Attempt to identify Trump (with 'known' data)

In [2]:
from pkg.mod import cluster
import numpy as np
from collections import Counter
from pandas import DataFrame
fields = ["n_hashtags", "n_mentions", "avg_syllables",
          "avg_word_length", "fk", 'n_sents',
          "n_ents", "n_uppers", "platform_id",'amplifier',
          'analneg', 'attribadj', 'auxdo', 'bemv',
          'bracket', 'caps', 'cconj', 'cntrstconj',
          'colon', 'comma', 'defart', 'detquan',
          'exclam', 'fstpp', 'fulstop', 'gerund',
          'havemv', 'imperative', 'indefart',
          'infinitive', 'it', 'mdnec',
          'mdposs', 'mdpred', 'multiwvb',
          'nomin', 'numdet', 'numnoun', 'objpro',
          'otheradv', 'othrintj', 'othrnoun',
          'othrverb', 'passive', 'past', 'perceptvb',
          'perfect', 'posesprpn', 'possdet', 'predadj',
          'prep', 'procontract', 'progressive',
          'proquan', 'provdo', 'prpn', 'prvv', 'pubv',
          'ques', 'relclausesubgap', 'sinflect',
          'sndpp', 'stancevb', 'subjpro', 'superlative',
          'thrdpp', 'timeadv', 'whw', 'initialmention']
dat = base.Data()
prep = base.Preprocessor()
dat.get_data(load_local=True)

prep.tokenize(data=dat.data)

ids_ = [d.ID for d in prep.docs]
arrs = [d.feature_array for d in prep.docs]
df = DataFrame(arrs, index=ids_, columns=fields)


100%|██████████| 17433/17433 [06:25<00:00, 45.24it/s]


Length Error: 
4746) Iran humiliated the United States with the capture of our 10 sailors. Horrible pictures and images. We are weak. I will NOT forget!
Length Error: 
4748) Iran humiliated the United States with the capture of our 10 sailors. Horrible pictures and images. We are weak. I will NOT forget!


In [3]:
tod_map = {}
dates, times, tokens = [], [], []
platforms = []
for d in prep.docs:
    dates.append(datetime.strptime(d.local_date, "%Y-%m-%d"))
    tm = d.local_time
    times.append(tm)
    platforms.append(d.platform)
    tokens.append(",".join(d.get_tokens_merged(lowercase=False)))
df.insert(0, "tokens", tokens)
df.insert(0, 'date', dates)
df.insert(0, 'time', times)
df.insert(0, 'platform', platforms)

In [4]:
from pkg.utils import make_tokens_list
from tabulate import tabulate

all_tokens = make_tokens_list(df['tokens'])
android_tokens_series = df.loc[[i for i in df.index
                                if df.loc[i]['platform']=='android']]['tokens']
android_tokens = make_tokens_list(android_tokens_series)
iphone_tokens_series = df.loc[[i for i in df.index
                               if df.loc[i]['platform']=='iphone']]['tokens']
iphone_tokens = make_tokens_list(iphone_tokens_series)


words_counter = dict(Counter(all_tokens).most_common())
android_counter = dict(Counter(android_tokens).most_common())
iphone_counter = dict(Counter(iphone_tokens).most_common())


android_words_unique = [(k, android_counter[k]) for k in android_counter.keys()
                        if k not in iphone_counter]

iphone_words_unique = [(kx, iphone_counter[kx]) for kx in iphone_counter.keys()
                       if kx not in android_counter]

sidexside = [(*android_words_unique[ix], *iphone_words_unique[ix]) for ix in range(40)]
combined_table = tabulate(sidexside, headers=("android_token", "android_count", 'iphone_token', 'iphone_count'))

print(combined_table)

android_token           android_count  iphone_token           iphone_count
--------------------  ---------------  -------------------  --------------
stupid                             52  Join                            145
Ebola                              41  Military                         41
Friends                            29  Cuts                             37
warming                            25  DACA                             36
Trump National Doral               24  prayers                          35
hell                               23  Video                            33
West Africa                        22  Fake News                        27
ENJOY                              20  Puerto Rico                      24
GLOBAL                             18  the White House                  21
WARMING                            18  Repeal                           21
shirts                             17  Anthem                           18
Celebrity Apprentice     

### Difference Between IPhone and Android

Its clear just from looking at the most common tokens that
the posts originating from Android are more "incendiary" in terms
of the insulting tone of many keywords. While this is certainly a characteristic
one might associate with Trump it is not enough by itself to support the
contention that Trump wrote all of these himself.

Ironically, it is the tokens from the IPhone group which discuss the "fake news"
in great detail. Though this makes sense if you consider that Trump switched to an
IPhone after taking office. So we expand our condition to include certain times of the day
in 2018 when he was suspected of tweeting and compare the counts again.

In [17]:
import time
from datetime import datetime

trump = df.loc[[ix for ix in df.index if (13 >= time.strptime(df.loc[ix]['time'], "%H:%M:%S").tm_hour >= 10
                                           and df.loc[ix]['date'].year == 2018
                                          and df.loc[ix]['platform']=='iphone')
                or df.loc[ix]['platform'] == 'android']]['tokens']

trump_tokens = make_tokens_list(trump)
not_trump = df.loc[[ix for ix in df.index if (time.strptime(df.loc[ix]['time'], "%H:%M:%S").tm_hour > 13 or time.strptime(df.loc[ix]['time'], "%H:%M:%S").tm_hour < 10)
                                           and df.loc[ix]['date'].year == 2018
                                          and df.loc[ix]['platform']=='iphone']]['tokens']

not_trump_tokens = make_tokens_list(not_trump)

trump_counter = dict(Counter(trump_tokens).most_common())
not_trump_counter = dict(Counter(not_trump_tokens).most_common())
trump_unique = [(tk, trump_counter[tk]) for tk in trump_counter if tk not in not_trump_counter]
not_trump_unique = [(ntk, not_trump_counter[ntk]) for ntk in not_trump_counter if ntk not in trump_counter]



sidexside2 = [(*trump_unique[ix], *not_trump_unique[ix]) for ix in range(min(len(trump_unique), len(not_trump_unique)))]
combined_table2 = tabulate(sidexside2, headers=("trump_token", "trump_count", 'nottrump_token', 'nottrump_count'))
print(combined_table2)


trump_token             trump_count  nottrump_token                        nottrump_count
--------------------  -------------  ----------------------------------  ----------------
tonight                         102  Military                                          17
Hillary                          82  Border                                             9
got                              75  Cuts                                               5
amazing                          70  Chrysler                                           5
debate                           53  opportunity                                        4
president                        52  Safety                                             4
stupid                           52  Shutdown                                           4
Ted Cruz                         52  Southern                                           4
China                            51  the White House                                    3
ObamaCare 

There were several things I noticed from the updated tokens lists:

    (1) The 'likely' Trump tokens are shorter on average.
    (2) The 'likely' Trump tokens are more sentiment oriented.
    (3) The 'unlikely' Trump tokens have more numeric values.

# Natural Groups

In [5]:
mod0 = cluster(data=arrs, cluster_meth='optics', xi=.000075,
               metric='l2', min_cluster_size=500)

mod0_labs = mod0.labels_
print(np.unique(mod0_labs))
print(dict(Counter(mod0_labs)))

print(len(mod0_labs))

df.insert(len(df.columns), "cluster", mod0_labs)

tod_map = {}
dates, times, tokens = [], [], []


OPTICS(metric='l2', min_cluster_size=500, xi=7.5e-05) Loaded.
[-1  0  1  2]
{-1: 15132, 2: 1094, 0: 584, 1: 507}
17317


### Combining filters

In [6]:
clusts = df['cluster']

outlier_filt = clusts == -1
clust_filt = clusts != -1

df_outliers = df.where(outlier_filt)
df_clusts = df.where(clust_filt)
df_outliers = df_outliers.dropna()
df_clusts = df_clusts.dropna()
grps = df_clusts.groupby(['cluster'])




In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from time import strptime
sns.set_theme(style="darkgrid")


fieldsx = fields.copy()
fieldsx.append('cluster')
fig = sns.pairplot(df[fieldsx], hue='cluster')

plt.show()
fig.savefig('plots/lang_attrs.png')

maybe_trump_df = df.loc[[idx for idx in df.index
                         if (df.loc[idx]['platform'] == 'android'
                             or df.loc[idx]['platform'] == 'web client')
                         or df.loc[idx]['date'] <=  datetime(2017, 1, 20)
                         or (df.loc[idx]['date'] > datetime(2017, 1, 20) and
                             13 > strptime(df.loc[idx]['time'], "%H:%M:%S").tm_hour > 10)
                         ]]

KeyboardInterrupt: 

### Conclusion

Ultimately, it is evident that text features alone cannot serve as a substitute
for observed patterns. However, it is interesting that the text features did segment
the posts into what could be considered rough categories where each group seems to be
centered on a theme. This would seem to indicate that, at least in this case, the linguistic
patterns of posts share features when they are about the topic or related to a theme.

Lastly, I have to give credit to ___ who wrote the article identifying the conditions
of Trumps tweets. I find there is a significant degree of merit in their observations.